In [0]:
%%sh
aws s3 ls s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI/version=1.0.0/range_type=MONTH/date=2019-08-31/
# --recursive --human --summarize | tail -4


In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/genre_report/version=1.0.0/range_type=MONTH/


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/granularity=monthly/

In [0]:
%%sh


In [0]:

raw_path = "s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI/version=1.0.0/range_type=MONTH/date=2019-08-31/"
df_raw = spark.read.parquet(raw_path)
df_raw.sort('value').show(3, True, True)


In [0]:

raw_path = "s3://aardvark-prod-pdx-mdm-to-int/genre_report/version=1.0.0/range_type=MONTH/"
# df_raw = spark.read.parquet(raw_path).where("NOT value=0")
df_raw = spark.read.parquet(raw_path)
df_raw.show(3, True, True)


In [0]:

uni_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/"
df_uni = spark.read.format('delta').load(uni_path)
df_uni.show(3, True, True)

In [0]:

# print "raw count = ", df_raw.count()
# print "raw dis count = ", df_raw.distinct().count()
# print "uni count = ", df_uni.count()
# print "uni dis count = ", df_uni.distinct().count()

# groupby
df_raw.groupby("metric_name").count().show()
print df_uni.count()

In [0]:

print df_raw.filter(" device_type='1' and platform='2'").count()
print df_uni.filter("_identifier like '12020032004%' and device_code='ios-phone'").count() * 2
# print df_uni.filter("device_code='ios-phone'").count()

In [0]:

print df_uni.where("date='2019-08-31'").count()
df_raw.where("date='2019-08-31'").groupby("metric_name").count().show()

In [0]:

df_uni.where("date LIKE '{}%'".format('2020-02')).show(1)

In [0]:
%angular
<h2>Transform Test

In [0]:

# Null value not exist for both est_cross_genre_affinity or est_cross_genre_usage_penetration 
df_uni.where("est_cross_genre_affinity is NULL or est_cross_genre_usage_penetration is NULL").take(1)

In [0]:

df_uni.where("not est_cross_genre_affinity > 0 or not est_cross_genre_affinity < 1000").show()

In [0]:

df_uni.printSchema()
df_raw.printSchema()

In [0]:

df_raw.where("selected_genre_id='202' and related_genre_id='204' and device_type='1' and country='HK' and date='2019-01-31' and platform='2'").show()
df_raw.where("selected_genre_id='204' and related_genre_id='202' and device_type='1' and country='HK' and date='2019-01-31' and platform='2'").show()
df_uni.where("cross_genre_id='202' and genre_id='204' and country_code='HK' and date='2019-01-31'").show()

In [0]:

df_uni.where("not est_cross_genre_usage_penetration > 0 or not est_cross_genre_usage_penetration <= 1").show()

In [0]:

row_uni = df_uni.collect()
for row in row_uni:
    if row['cross_genre_id'] == row['genre_id']:
        raise RuntimeError("Equal")

print "PASS"

In [0]:

row_raw = df_raw.groupby("metric_name").count().collect()
uni_count = df_uni.count()
for row in row_raw:
    print row["count"] == uni_count

In [0]:

COUNTRIES = ['TR', 'RU', 'FR', 'KR', 'TW', 'CA', 'DE', 'JP', 'IT', 'US', 'HK', 'AU', 'GB', 'BR', 'MX', 'ES']
fail_list = {i: False for i in COUNTRIES}

row_uni = df_uni.groupby("country_code").count().collect()
row_raw = df_raw.groupby("country").count().collect()

for row1 in row_uni:
    for row2 in row_raw:
        if row1['count'] * 2 == row2['count']:
            fail_list[row1['country_code']] = True

print fail_list

In [0]:

df_uni_country = df_uni.groupby("country_code").count().withColumnRenamed("country_code", "country").withColumnRenamed("count", "count_uni")
df_raw_country = df_raw.groupby("country").count().withColumnRenamed("count", "count_raw")

df_join = df_uni_country.join(df_raw_country, "country")
df_join.show()

In [0]:

print u"cannot resolve '`country_code`' given input columns: [aa];;\n'Aggregate ['country_code], ['country_code, count(1) AS count#803L]\n+- Project [country_code#620 AS aa#799]\n   +- Relation[_identifier#619L,country_code#620,cross_genre_id#621,date#622,device_code#623,est_cross_genre_affinity#624,est_cross_genre_usage_penetration#625,genre_id#626,granularity#627] parquet\n"

In [0]:

DEVICES = [('ios-phone', '2'), ('android-phone', '1')]

fail_list = []

df_uni_device = df_uni.groupby("device_code").count()
df_raw_device = df_raw.groupby("platform").count()

for device_code, platform in DEVICES:
    uni_count = df_uni_device.where("device_code='{}'".format(device_code)).collect()[0][1]
    raw_count = df_raw_device.where("platform='{}'".format(platform)).collect()[0][1]
    if not uni_count == raw_count / 2:
        print "uni_count = ", uni_count
        print "raw_count = ", raw_count
        fail_list.append((device_code, platform))

print fail_list

In [0]:

import datetime
from dateutil.relativedelta import *

start_date = datetime.datetime(2019, 1, 31)

now_date = datetime.datetime.now() - relativedelta(months=+1)
now_date = now_date.strftime("%Y-%m-%d")
print now_date

temp_date = start_date.strftime("%Y-%m-%d")
while temp_date < now_date:
    print temp_date[:7]
    if not len(df_uni.where("date like '{}%'".format(temp_date[:7])).collect()):
        print temp_date[:7]
    
    start_date += relativedelta(months=+1)
    temp_date = start_date.strftime("%Y-%m-%d")

print "PASS"

In [0]:

COUNTRY_CODE = set(['TR', 'RU', 'FR', 'KR', 'TW', 'CA', 'DE', 'JP', 'IT', 'US', 'HK', 'AU', 'GB', 'BR', 'MX', 'ES'])

temp = set([])
for row in df_uni.groupby("country_code").count().collect():
    temp.add(row['country_code'])

if temp == COUNTRY_CODE:
    print "PASS"


In [0]:

import psycopg2
import datetime

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_citus_db",
        user="citus_bdp_usage_qa",
        host="10.2.10.132",
        password="dNzWtSV3pKTx",
        port=5432
    )
)
# sql = "SELECT * FROM pg_class WHERE relname LIKE 'usage_genre_cross_fact_v1%';"
# sql = "SELECT * FROM usage.usage_genre_cross_fact_v1_p_201909 LIMIT 1"
# sql = "SELECT * FROM pg_indexes WHERE schemaname = 'usage' AND tablename LIKE 'usage_genre_cross_fact_v1%'"

In [0]:

sql_1 = "SELECT * FROM usage.usage_genre_cross_fact_v1 WHERE est_cross_genre_affinity is NULL or est_cross_genre_usage_penetration is NULL"
res_1 = query(citus_dsn_, sql_1)
print len(res_1)

In [0]:

sql_2 = "SELECT * FROM usage.usage_genre_cross_fact_v1 WHERE est_cross_genre_affinity <= 0"
res_2 = query(citus_dsn_, sql_2)
print len(res_2)
print res_2

In [0]:

sql_3 = "SELECT * FROM usage.usage_genre_cross_fact_v1 WHERE est_cross_genre_usage_penetration <= 0 or est_cross_genre_usage_penetration > 1"
res_3 = query(citus_dsn_, sql_3)
print len(res_3)
print res_3

In [0]:

# db
sql_4 = "SELECT count(*) FROM usage.usage_genre_cross_fact_v1"
res_4 = query(citus_dsn_, sql_4)
print res_4[0][0]

# raw
print df_raw.count() / 2 == res_4[0][0]

In [0]:

COUNTRIES = ['TR', 'RU', 'FR', 'KR', 'TW', 'CA', 'DE', 'JP', 'IT', 'US', 'HK', 'AU', 'GB', 'BR', 'MX', 'ES']

sql_5 = "SELECT country_code, count(*) FROM usage.usage_genre_cross_fact_v1 GROUP BY country_code"
res_5 = query(citus_dsn_, sql_5)
print len(res_5)
# print res_5
# print row_raw

db_fail_list = {i:False for i in COUNTRIES}
for tup in res_5:
    for row in row_raw:
        if tup[1] * 2 == row['count']:
            db_fail_list[tup[0]] = True

print db_fail_list

for a, b in db_fail_list.items():
    if not b:
        raise RuntimeError("Failed")
print 'PASS'

In [0]:

DEVICES = [('ios-phone', '2'), ('android-phone', '1')]

sql_6 = "SELECT device_code, count(*) FROM usage.usage_genre_cross_fact_v1 GROUP BY device_code"
res_6 = query(citus_dsn_, sql_6)
print len(res_6)
# print res_6
print df_raw_device.collect()

db_fail_list = {i:False for i in DEVICES}
for tup in res_6:
    for row in df_raw_device.collect():
        if tup[1] * 2 == row['count']:
            db_fail_list[tup[0]] = True

print db_fail_list

for a, b in db_fail_list.items():
    if not b:
        raise RuntimeError("Failed")
print 'PASS'

In [0]:

import datetime
from dateutil.relativedelta import *

sql_7 = "SELECT date, count(*) FROM usage.usage_genre_cross_fact_v1 GROUP BY date"
res_7 = query(citus_dsn_, sql_7)
print res_7

start_date = datetime.datetime(2019, 1, 31)
start_date += relativedelta(months=+1)

now_date = datetime.datetime.utcnow().strftime("%Y-%m-%d")

temp_date = start_date.strftime("%Y-%m-%d")
while temp_date < now_date:
    sql = "SELECT * FROM usage.usage_genre_cross_fact_v1 AS t1 WHERE date::text LIKE '{}%'".format(temp_date[:7])
    temp = query(citus_dsn_, sql)
    if not len(temp):
        raise RuntimeError("FAIL")
    
    start_date += relativedelta(months=+1)
    temp_date = start_date.strftime("%Y-%m-%d")

print "PASS"

In [0]:

from decimal import *

PRECISION = 6

sql_8 = "SELECT * FROM usage.usage_genre_cross_fact_v1"
res_8 = query(citus_dsn_, sql_8)

prec_fail_list = []
print len(res_8)
for i in range(len(res_8)):
    prec_1 = -res_8[i][6].as_tuple().exponent
    prec_2 = -res_8[i][7].as_tuple().exponent
    if not prec_1 == PRECISION or not prec_2 == PRECISION:
        prec_fail_list.append(res_8[i])

if not prec_fail_list:
    print "PASS"
else:
    print len(prec_fail_list)

In [0]:

from decimal import *

sql = "SELECT * FROM usage.usage_genre_cross_fact_v1 AS t1 WHERE date::text LIKE '2019-02-%' LIMIT 2"
a = query(citus_dsn_, sql)
print a
-a[0][6].as_tuple().exponent


In [0]:
%%sh
# SELECT *  FROM   advancedreview_topic_fact_v1 where date='2019-07-08' and country_code='AU' limit 4;

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=usage;
BEGIN;
SELECT * FROM usage.usage_genre_cross_fact_v1 WHERE est_cross_genre_affinity IS NULL or est_cross_genre_usage_penetration IS NULL;

-- SELECT * FROM usage.usage_genre_cross_fact_v1 WHERE date::text LIKE '2020-02%' LIMIT 1;
-- SELECT * FROM usage.usage_genre_cross_fact_v1 AS t1 WHERE date::text LIKE '2019-02-%' LIMIT 2;
-- SELECT * FROM usage.usage_genre_cross_fact_v1 LIMIT 2;
-- SELECT device_code, count(*) FROM usage.usage_genre_cross_fact_v1 GROUP BY device_code;
\d usage.usage_genre_cross_fact_v1_p_201907
-- EXPLAIN ANALYZE SELECT * FROM  usage_genre_cross_fact_v1 where granularity='monthly' and device_code='ios-phone'  and country_code='RU'  and date='2019-08-31' and genre_id='204' ;
SELECT * FROM  usage_genre_cross_fact_v1 LIMIT 10;
-- SELECT max(est_cross_genre_affinity) , min(est_cross_genre_affinity) FROM  usage_genre_cross_fact_v1 where granularity='monthly'   ;
-- 
-- SELECT * FROM  usage_genre_cross_fact_v1 where granularity='monthly' and cast(_identifier as varchar)  like '12020032004%' order by  date desc limit 3;
-- COMMIT;
-- 
-- SELECT *
-- FROM
--     pg_indexes
-- WHERE 
--     schemaname = 'usage' AND tablename LIKE 'usage_genre_cross_latest%'
-- 
-- ;

-- SELECT schema_name FROM information_schema.schemata;
-- SELECT nspname FROM pg_catalog.pg_namespace;

-- SELECT * FROM pg_class WHERE relname LIKE 'usage_genre_cross_fact_v1%';
-- COMMIT;

SELECT * FROM usage.usage_genre_cross_fact_v1 WHERE est_cross_genre_affinity = 0;

EOF



In [0]:
%%sh


In [0]:

raise RuntimeError("zz")

In [0]:

len(row_uni)

In [0]:

from pyspark.sql import functions as F
# df_raw_after_drop = df_raw.drop("metric_name", "value", "device_type")
df_raw.show(3)

# df_raw_after_drop.withColumn("platform", df_raw_after_drop.when(df_raw_after_drop.platform = 2, "ios-phone").otherwise("android-phone"))
# , F.when(df_raw.platform=2, "ios-phone").when(df_raw.platform=1, "android-phone")
df_raw_new = df_raw.select(df_raw.selected_genre_id, df_raw.related_genre_id, df_raw.country, \
                            F.when(df_raw.platform==2, "ios-phone").when(df_raw.platform==1, "android-phone").alias("device_code"))
df_raw_new.show(3)

In [0]:

df_uni.groupby("country_code").count().show(100, True, True)


In [0]:

df_uni.groupby("device_code").count().show(100, True, True)

In [0]:

df_uni.where("est_cross_genre_affinity<=0").show(100, True, True)

In [0]:

df_uni.where("country_code is null or device_code is null or cross_genre_id is null or genre_id is null or est_cross_genre_affinity is null or est_cross_genre_usage_penetration is null").show()

In [0]:

df_raw.groupby("metric_name").count().show()

In [0]:

df_raw_filter_by_metric = df_raw.where("metric_name='XUP' or metric_name='XGA'")
df_uni_filter_by_metric = df_uni.where("metric_name='XUP' or metric_name='XGA'")
print 'raw', df_raw_filter_by_metric.count()
print 'uni', df_uni_filter_by_metric.count()
df_subtract_filtered = df_raw_filter_by_metric.subtract(df_uni_filter_by_metric)
print 'sub', df_subtract_filtered.count()

In [0]:

print 'dis raw', df_raw_filter_by_metric.distinct().count()

In [0]:

df_uni.printSchema()
df_raw.printSchema()

In [0]:
%%sh
pip install pyspark

In [0]:
%%sh
which pyspark
cd /usr/bin/
ls

In [0]:
%%sh
aws s3 cp s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/_delta_log/00000000000000000013.json - | head

In [0]:

import psycopg2
import datetime

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_citus_db",
        user="citus_bdp_usage_qa",
        host="10.2.10.132",
        password="dNzWtSV3pKTx",
        port=5432
    )
)


In [0]:

import unittest
import datetime
from dateutil.relativedelta import *
from applications.db_check_v1.common.base_test import PipelineTest
# from aadatapipelinecore.core.utils.encode import activate_system_utf8
from aadatapipelinecore.core.utils.spark import create_spark
# from applications.db_check_v1.common.constants import query, citus_settings
from applications.db_check_v1.common.db_check_utils import etl_skip



class GameIQCrossGenreRawData(object):
    """
    Get data from Data Foundation
    """
    _raw_s3_path = "s3://aardvark-prod-pdx-mdm-to-int/genre_report/version=1.0.0/range_type=MONTH/"

    def __init__(self, spark):
        self.spark = spark

    def get(self, condition=None):
        """
        :return: Raw data from DF team
        :rtype: pyspark.sql.DataFrame
        """
        # Filter value = 0 data (confirmed with DF team)
        raw_data = self.spark.read.parquet(self._raw_s3_path).where("NOT value=0")
        if condition:
            raw_data = raw_data.where(condition)

        return raw_data


class GameIQCrossGenreUnifiedData(object):
    """
    Get data after ETL process
    """
    _uni_s3_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/"

    def __init__(self, spark):
        self.spark = spark

    def get(self, condition=None):
        """
        :return: Unified data (after ETL)
        :rtype: pyspark.sql.DataFrame
        """
        uni_data = self.spark.read.format('delta').load(self._uni_s3_path)
        if condition:
            uni_data = uni_data.where(condition)

        return uni_data
        



class TestGameIQCrossGenre(PipelineTest):
    trigger_date_config = ('0 5 7 * *', 7)
    COUNTRIES = ['TR', 'RU', 'FR', 'KR', 'TW', 'CA', 'DE', 'JP', 'IT', 'US', 'HK', 'AU', 'GB', 'BR', 'MX', 'ES']
    DEVICES = [('ios-phone', '2'), ('android-phone', '1')]

    def setUp(self):
        super(TestGameIQCrossGenre, self).setUp()
        print self.check_date_str
        self.df_raw = GameIQCrossGenreRawData(self.spark).get(
            "date LIKE '{}%'".format(self.check_date_str[:7])  # Get only month
        )
        self.df_uni = GameIQCrossGenreUnifiedData(self.spark).get(
            "date LIKE '{}%'".format(self.check_date_str[:7])
        )
        # self.df_raw = GameIQCrossGenreRawData(self.spark).get()
        # self.df_uni = GameIQCrossGenreUnifiedData(self.spark).get()


    @classmethod
    def setUpClass(cls):
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext

    @etl_skip()
    def test_game_iq_cross_genre_db_data_date_range(self):
        """
        This is an regression test for all date range
        Date range (2019-01-01 ~ check_date)
        """
        fail_list = []
        start_date = datetime.datetime(2019, 1, 31)
        temp_date = start_date.strftime("%Y-%m-%d")
        while temp_date <= self.check_date_str:
            sql = "SELECT * FROM usage.usage_genre_cross_fact_v1 " \
                  "WHERE date::text LIKE '{}%' LIMIT 1".format(temp_date[:7])
            if not query(citus_dsn_, sql):
                fail_list.append(temp_date[:7])

            start_date += relativedelta(months=+1)
            temp_date = start_date.strftime("%Y-%m-%d")

        self.assertEqual(fail_list, [], msg="Expected date {} doesn't exist".format(fail_list))


unittest.main(argv=[''], verbosity=2)
%tb


In [0]:

import croniter
import datetime

def get_date(trigger_datetime):
    schedule, days_delta = ('* * 27 * *', 27)
    cron = croniter.croniter(schedule, trigger_datetime)
    date = cron.get_prev(datetime.datetime) - datetime.timedelta(days=days_delta)
    return date
    
print get_date(datetime.datetime(2019, 4, 1))

In [0]:

total = query(citus_dsn_, 
    "SELECT count(*) "
    "FROM usage.usage_genre_cross_fact_v1 "
    "WHERE date::text LIKE '{}%'".format('2020-02')
)
print total

In [0]:

import sys
import datetime
import traceback
import unittest

def debug(case_list):
    std_out_origin= sys.stdout
    std_err_origin= sys.stderr
    try:
        suite =  unittest.TestSuite()
        for case in case_list:
            suite.addTest(case)
        runner = unittest.TextTestRunner(verbosity=2, buffer=True)
        runner.run(suite)
    except Exception as ex:
        print dir(ex)
        print ex.message
        traceback.print_exception(type(ex), ex, ex.__traceback__)
    finally:
        sys.stdout = std_out_origin
        sys.stderr = std_err_origin
    
case_name_list = [
    "test_game_iq_cross_genre_no_null_value"
    ]


test_case_list = [] # pass 
test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestGameIQCrossGenre))



debug(test_case_list)

In [0]:

a = GameIQCrossGenreRawData(spark).get("date LIKE '{}%'".format('2020-02'))
a.select("date").groupby("date").count().show()

In [0]:

from applications.db_check_v1.common.constants import query, citus_settings

In [0]:
%python
